## Data Ingestion

In [1]:
import os
os.chdir('../')
%pwd

'/home/utpal108/dev/Python/PW_Skills_Projects/Home-Loan-Approval-Prediction'

In [2]:
# Config Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    source_url: str
    raw_dataset_dir: Path
    raw_data_path: str
    train_data_path: Path
    test_data_path: Path

In [3]:
from homeLoan.constants import *
from homeLoan.utils import create_directories, read_yaml, save_object

In [4]:
# Configuration Manager
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir, config.raw_dataset_dir, config.dataset_dir])

        data_ingestion_config = DataIngestionConfig(
            source_url = config.source_URL,
            raw_dataset_dir = Path(config.raw_dataset_dir),
            raw_data_path = config.raw_data_path,
            train_data_path = Path(config.train_data_path),
            test_data_path = Path(config.test_data_path)
        )

        return data_ingestion_config

In [5]:
import pandas as pd
import gdown
from sklearn.model_selection import train_test_split

In [6]:
# Components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def _download_dataset(self) -> None:
        '''
        Fetch the dataset from the URL
        '''
        try:
            if(not os.listdir(self.config.raw_dataset_dir)):
                dataset_url = self.config.source_url
                output_path = self.config.raw_data_path
                file_id = dataset_url.split('/')[-2]
                prefix = 'https://drive.google.com/uc?/export=download&id='
                gdown.download(prefix+file_id, output_path)

        except Exception as e:
            raise e

    def initiate_data_ingestion(self) -> None:
        
        try:
            self._download_dataset()
            df = pd.read_csv(self.config.raw_data_path)
            train_df, test_df = train_test_split(df, test_size=0.20, random_state=42, shuffle=True)
            train_df.to_csv(self.config.train_data_path, columns=df.columns, index=False)
            test_df.to_csv(self.config.test_data_path, columns=df.columns, index=False)

        except Exception as e:
            raise e
    

In [7]:
# Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.initiate_data_ingestion()

except Exception as e:
    raise e

2023-12-02 22:16:29,878 : homeLoan.logger - INFO - YAML file: config/config.yaml loaded successfully
2023-12-02 22:16:29,881 : homeLoan.logger - INFO - YAML file: params.yaml loaded successfully
2023-12-02 22:16:29,883 : homeLoan.logger - INFO - created directory at: artifacts
2023-12-02 22:16:29,884 : homeLoan.logger - INFO - created directory at: artifacts/data_ingestion
2023-12-02 22:16:29,886 : homeLoan.logger - INFO - created directory at: artifacts/data_ingestion/raw_dataset
2023-12-02 22:16:29,888 : homeLoan.logger - INFO - created directory at: artifacts/data_ingestion/dataset
